In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange
from IPython.display import Video

from nmf_grooming import (
    NeuromechflyGrooming,
)

from flygym.mujoco import Parameters, NeuroMechFly

In [ ]:
# Define the joint names
LH_COXA = 3
LH_COXA_YAW = 4
LH_COXA_ROLL = 5
LH_FEMUR = 6
LH_FEMUR_ROLL = 7
LH_TIBIA = 8
LH_TARSUS = 9

RH_COXA = 10
RH_COXA_YAW = 11
RH_COXA_ROLL = 12
RH_FEMUR = 13
RH_FEMUR_ROLL = 14
RH_TIBIA = 15
RH_TARSUS = 16

L_PEDICEL = 17
L_PEDICEL_YAW = 18

R_PEDICEL = 19
R_PEDICEL_YAW = 20

A1A2 = 21
A3 = 22
A4 = 23
A5 = 24
A6 = 25

NB_JOINTS = 26
NB_ABD_JOINTS = 5

##### Load the grooming data

In [ ]:
grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
with open(grooming_module_path, "rb") as f:
    grooming_modules = pickle.load(f)

timestep = grooming_modules["timestep"]

#### Foreleg joint position (not symmetrical)

In [ ]:
sim_params = Parameters(
    timestep=timestep,
    render_mode="saved",
    render_playspeed=0.1,
    draw_contacts=True,
    render_camera="Animat/camera_front",
)

target_joint_angles_front = grooming_modules["foreleg"]
seq_length = len(grooming_modules['foreleg'][0])

abd_gain = 0.3
nmf = NeuromechflyGrooming(
    sim_params=sim_params,
)

target_joint_angles_front = grooming_modules["foreleg"]
abd_arr = abd_gain*np.ones((NB_ABD_JOINTS, seq_length))
target_joint_angles_front = np.concatenate([target_joint_angles_front, abd_arr], axis=0)

nbre_step = seq_length-1

# 10 loops of frontleg grooming
replay_steps = [nbre_step,2*nbre_step,3*nbre_step,4*nbre_step,5*nbre_step,
                6*nbre_step,7*nbre_step,8*nbre_step,9*nbre_step,10*nbre_step]

joint_pos = np.zeros(26)
k = 0
nmf.reset()
for i in trange(10*seq_length):
    joint_pos = target_joint_angles_front[:, k]
    
    action = {"joints": joint_pos}
    nmf.step(action)
    nmf.render()
    if i in replay_steps:
        k = 0
    else:
        k += 1
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "frontleg_groom_front.mp4"
else:
    frontleg_video = "frontleg_groom_left.mp4"  

nmf.save_video(frontleg_video)

#### Get symetrical movement

In [ ]:
idx_diff_LR = 7 # index difference between the right and left leg joints
symetric_leg = np.zeros_like(target_joint_angles_front) # contains all the joint angles for front leg grooming symmetrically

for i in range(len(target_joint_angles_front)):
    if i in [RH_COXA, RH_FEMUR, RH_TIBIA, RH_TARSUS]:
        symetric_leg[i,:] = target_joint_angles_front[i-idx_diff_LR,:] # map the right pitch joints to the left ones
    elif i in [RH_COXA_YAW, RH_COXA_ROLL, RH_FEMUR_ROLL]:
        symetric_leg[i,:] = -target_joint_angles_front[i-idx_diff_LR,:] # map the right yaw roll joints to the left ones
    else:
        symetric_leg[i,:] = target_joint_angles_front[i,:]


##### Delay the right leg movement / joint position

In [ ]:
def delay_sequence(seq, start_row, end_row, delay):
    # Extract rows to be delayed
    shifted_seq = seq.copy()
    rows_to_shift = seq[start_row:end_row+1,:]

    # Extract columns
    new_col = rows_to_shift[:, delay:]
    first_columns = rows_to_shift[:, :delay]

    shifted_columns = np.concatenate([new_col, first_columns], axis=1)
    shifted_seq[start_row:end_row+1,:] = shifted_columns
    return shifted_seq

In [ ]:
initial_sequence = symetric_leg.copy()
rep = 5
delay = seq_length//2
repeated_sequence = np.tile(initial_sequence, (1, rep))

delayed_sequence = delay_sequence(repeated_sequence, RH_COXA, RH_TARSUS, delay)

#### Front leg grooming symmetrical

In [ ]:
sim_params = Parameters(
    timestep=timestep,
    render_mode="saved",
    render_playspeed=0.1,
    draw_contacts=True,
    render_camera="Animat/camera_front",
)

nmf = NeuromechflyGrooming(
    sim_params=sim_params,
)

joint_pos = np.zeros(NB_JOINTS)

nmf.reset()
for i in trange(rep*seq_length):
    joint_pos = delayed_sequence[:, i]
    action = {"joints": joint_pos}
    nmf.step(action)
    nmf.render()
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "frontleg_sym_front.mp4"
elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "frontleg_sym_left.mp4"
elif sim_params.render_camera == "Animat/camera_right":
    frontleg_video = "frontleg_sym_right.mp4"
else:
    frontleg_video = "frontleg_sym_back.mp4"  

nmf.save_video(frontleg_video)

#### Hindleg grooming symmetrical

In [ ]:
sim_params = Parameters(
    timestep=timestep,
    render_mode="saved",
    render_playspeed=0.1,
    draw_contacts=True,
    render_camera="Animat/camera_back",
)

nmf = NeuromechflyGrooming(
    sim_params=sim_params,
)

target_joint_angles_back = symetric_leg

seq_length = len(target_joint_angles_back[0])
nbre_step = seq_length-1

# 10 loops of frontleg grooming
replay_steps = [nbre_step,2*nbre_step,3*nbre_step,4*nbre_step,5*nbre_step,
                6*nbre_step,7*nbre_step,8*nbre_step,9*nbre_step,10*nbre_step]

joint_pos = np.zeros(NB_JOINTS)
k = 0
nmf.reset()
for i in trange(10*seq_length):
    for j in range(NB_JOINTS):
        if j == LH_COXA or j == RH_COXA: 
            joint_pos[j] = target_joint_angles_back[j, k] + np.pi/12
        elif j == LH_FEMUR or j == RH_FEMUR:
            joint_pos[j] = target_joint_angles_back[j, k] + 7*np.pi/6
        elif j == LH_TIBIA or j == RH_TIBIA: # Tibia 
            joint_pos[j] = target_joint_angles_back[j, k] - 6*np.pi/7
        elif j == LH_TARSUS or j == RH_TARSUS: 
            joint_pos[j] = target_joint_angles_back[j, k] + 2*np.pi/3
        else:
            joint_pos[j] = target_joint_angles_back[j, k]
    
    action = {"joints": joint_pos}
    nmf.step(action)
    nmf.render()

    if i in replay_steps:
        k = 0
    else:
        k += 1    
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "hindleg_sym_front.mp4"

elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "hindleg_sym_left.mp4"

else:
    frontleg_video = "hindleg_sym_back.mp4"  
    
nmf.save_video(frontleg_video)

### Hindleg grooming

1. Raise the abdomen and the hindlegs to get in grooming position

In [ ]:
sim_params = Parameters(
    timestep=timestep,
    render_mode="saved",
    render_playspeed=0.1,
    draw_contacts=True,
    render_camera="Animat/camera_left",
)

nmf = NeuromechflyGrooming(
    sim_params=sim_params,
)
abd_gain = 0.3
abd_up_arr = np.zeros([NB_JOINTS, rep*seq_length])

# Set the hind legs to the initial position for grooming
for j in range(NB_JOINTS):
    if j == LH_COXA or j == RH_COXA:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0], rep*seq_length)
    elif j == LH_FEMUR or j == RH_FEMUR:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0] + 7*np.pi/6, rep*seq_length)
    elif j == LH_FEMUR_ROLL:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0] - np.pi/6, rep*seq_length)
    elif j == RH_FEMUR_ROLL:
        abd_up_arr[j,:] = np.linspace(0, -delayed_sequence[j,0] - np.pi/6, rep*seq_length)
    elif j == LH_TIBIA or j == RH_TIBIA:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0] - 2*np.pi/3, rep*seq_length)
    elif j == LH_TARSUS or j == RH_TARSUS:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0] + np.pi/2, rep*seq_length)
    elif j in [A1A2, A3, A4, A5, A6]:
        abd_up_arr[j,:(rep*seq_length)//2] = np.linspace(0, abd_gain, rep*seq_length//2)
        abd_up_arr[j,(rep*seq_length)//2:] = np.ones((rep*seq_length)//2)*(abd_gain)
    else:
        abd_up_arr[j,:] = np.linspace(0, delayed_sequence[j,0], rep*seq_length)
   

joint_pos = np.zeros(NB_JOINTS)

nmf.reset()
for i in trange(rep*seq_length):
    joint_pos = abd_up_arr[:, i]
    action = {"joints": joint_pos}
    nmf.step(action)
    nmf.render()

last_abd_up = abd_up_arr[:, -1]

if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "abd_up_front.mp4"

elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "abd_up_left.mp4"
elif sim_params.render_camera == "Animat/camera_back":
    frontleg_video = "abd_up_back.mp4"
else:
    frontleg_video = "abd_up.mp4"  
    
nmf.save_video(frontleg_video)


2. Hindleg Grooming

In [ ]:
target_joint_angles_back = delayed_sequence.copy()

joint_pos = np.zeros(NB_JOINTS)

# nmf.reset()
for i in trange(rep*seq_length):
    for j in range(NB_JOINTS):
        if j == LH_COXA or j == RH_COXA:
            joint_pos[j] = target_joint_angles_back[j, i]
        elif j == LH_COXA_ROLL or j == RH_COXA_ROLL:
            joint_pos[j] = target_joint_angles_back[j, i] - np.pi/6
        elif j == LH_FEMUR or j == RH_FEMUR: # Femur
            joint_pos[j] = target_joint_angles_back[j, i] + 7*np.pi/6
        elif j == LH_FEMUR_ROLL:
            joint_pos[j] = target_joint_angles_back[j, i] - np.pi/6
        elif j == RH_FEMUR_ROLL:
            joint_pos[j] = -target_joint_angles_back[j, i] - np.pi/6
        elif j == LH_TIBIA or j == RH_TIBIA: # Tibia 
            joint_pos[j] = target_joint_angles_back[j, i] - 2*np.pi/3
        elif j == LH_TARSUS or j == RH_TARSUS: # Tarsus
            joint_pos[j] = target_joint_angles_back[j, i] + np.pi/2
        elif j in [A1A2, A3, A4, A5, A6]:
            joint_pos[j] = last_abd_up[j]
        else:
            joint_pos[j] = 0
    
    action = {"joints": joint_pos}
    nmf.step(action)
    nmf.render()
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "hindleg_groom_front.mp4"

elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "hindleg_groom_left.mp4"

else:
    frontleg_video = "hindleg_groom_back.mp4"  
    
nmf.save_video(frontleg_video)

In [ ]:
Video(frontleg_video)

In [ ]:
# Plot the joint angles for the hind leg grooming

plt.figure(figsize=(11, 6))
plt.plot(target_joint_angles_back[LH_COXA,:], label='left coxa')
plt.plot(target_joint_angles_back[RH_COXA,:], label='right coxa')
plt.plot(target_joint_angles_back[LH_FEMUR,:], label='left femur')
plt.plot(target_joint_angles_back[RH_FEMUR,:], label='right femur')
plt.plot(target_joint_angles_back[LH_TIBIA,:], label='left tibia')
plt.plot(target_joint_angles_back[RH_TIBIA,:], label='right tibia')
plt.plot(target_joint_angles_back[LH_TARSUS,:], label='left tarsus')
plt.plot(target_joint_angles_back[RH_TARSUS,:], label='right tarsus')
plt.plot(target_joint_angles_back[LH_COXA_YAW,:], label='left coxa yaw')
plt.plot(target_joint_angles_back[RH_COXA_YAW,:], label='right coxa yaw')
plt.plot(target_joint_angles_back[LH_COXA_ROLL,:], label='left coxa roll')
plt.plot(target_joint_angles_back[RH_COXA_ROLL,:], label='right coxa roll')
plt.plot(target_joint_angles_back[LH_FEMUR_ROLL,:], label='left femur roll')
plt.plot(target_joint_angles_back[RH_FEMUR_ROLL,:], label='right femur roll')
plt.title('Joint positions for hind leg grooming')
plt.legend()
plt.xlabel('Time step (1 step = 0.001 s)')
plt.ylabel('Joint angle (rad)')
plt.grid()


### Plot joints position during frontleg grooming

In [ ]:
import matplotlib.pyplot as plt

target_joint_angles_repeat = np.concatenate((target_joint_angles_front, target_joint_angles_front), axis=1)
target_joint_angles_repeat = np.concatenate((target_joint_angles_repeat, target_joint_angles_repeat), axis=1)
target_joint_angles_repeat = np.concatenate((target_joint_angles_repeat, target_joint_angles_repeat), axis=1)


plt.figure(figsize=(15, 7))

#### Femur ####
fem_color_pitch = 'purple'
fem_color_roll = 'black'
min_femur_pitch, min_id_fem_pitch = target_joint_angles_repeat[LH_FEMUR,:].min(), target_joint_angles_repeat[LH_FEMUR,:].argmin()
max_femur_pitch, max_id_fem_pitch = target_joint_angles_repeat[LH_FEMUR,:].max(), target_joint_angles_repeat[LH_FEMUR,:].argmax()
min_femur_roll, min_id_fem_roll = target_joint_angles_repeat[LH_FEMUR_ROLL,:].min(), target_joint_angles_repeat[LH_FEMUR_ROLL,:].argmin()
max_femur_roll, max_id_fem_roll = target_joint_angles_repeat[LH_FEMUR_ROLL,:].max(), target_joint_angles_repeat[LH_FEMUR_ROLL,:].argmax()


# plot femur pitch, min and max
plt.plot(target_joint_angles_repeat[LH_FEMUR,:], label='L Femur Pitch', color=fem_color_pitch)
plt.scatter(min_id_fem_pitch, min_femur_pitch, color=fem_color_pitch, label='Min 'f'{min_id_fem_pitch:.2f}')
plt.scatter(max_id_fem_pitch, max_femur_pitch, color=fem_color_pitch, label='Max 'f'{max_id_fem_pitch:.2f}')


# plot femur roll, min and max
plt.plot(target_joint_angles_repeat[LH_FEMUR_ROLL,:], label='L Femur Roll', color=fem_color_roll)
plt.scatter(min_id_fem_roll, min_femur_roll, color=fem_color_roll, label='Min 'f'{min_id_fem_roll:.2f}')
plt.scatter(max_id_fem_roll, max_femur_roll, color=fem_color_roll, label='Max 'f'{max_id_fem_roll:.2f}')

#### Tibia ####
tibia_color = 'cyan'
min_tibia_pitch, min_id_tibia_pitch = target_joint_angles_repeat[LH_TIBIA,:].min(), target_joint_angles_repeat[LH_TIBIA,:].argmin()
max_tibia_pitch, max_id_tibia_pitch = target_joint_angles_repeat[LH_TIBIA,:].max(), target_joint_angles_repeat[LH_TIBIA,:].argmax()

# plot tibia pitch
plt.plot(target_joint_angles_repeat[LH_TIBIA,:], label='L Tibia Pitch', color=tibia_color)

# plot min & max of tibia pitch
plt.scatter(min_id_tibia_pitch, min_tibia_pitch, color=tibia_color, label='Min 'f'{min_id_tibia_pitch:.2f}')
plt.scatter(max_id_tibia_pitch, max_tibia_pitch, color=tibia_color, label='Max 'f'{max_id_tibia_pitch:.2f}')

#### Tarsus ####
tarsus_color = 'orange'
min_tarsus_pitch, min_id_tarsus_pitch = target_joint_angles_repeat[LH_TARSUS,:].min(), target_joint_angles_repeat[LH_TARSUS,:].argmin()
max_tarsus_pitch, max_id_tarsus_pitch = target_joint_angles_repeat[LH_TARSUS,:].max(), target_joint_angles_repeat[LH_TARSUS,:].argmax()

# plot tarsus pitch
plt.plot(target_joint_angles_repeat[LH_TARSUS,:], label='L Tarsus Pitch', color=tarsus_color)

plt.scatter(min_id_tarsus_pitch, min_tarsus_pitch, color=tarsus_color, label='Min 'f'{min_id_tarsus_pitch:.2f}')
plt.scatter(max_id_tarsus_pitch, max_tarsus_pitch, color=tarsus_color, label='Max 'f'{max_id_tarsus_pitch:.2f}')


##################### RIGHT LEG #####################

Rfem_color_pitch = 'magenta'
Rfem_color_roll = 'navy'
Rmin_femur_pitch, Rmin_id_fem_pitch = target_joint_angles_repeat[RH_FEMUR,:].min(), target_joint_angles_repeat[RH_FEMUR,:].argmin()
Rmax_femur_pitch, Rmax_id_fem_pitch = target_joint_angles_repeat[RH_FEMUR,:].max(), target_joint_angles_repeat[RH_FEMUR,:].argmax()
Rmin_femur_roll, Rmin_id_fem_roll = target_joint_angles_repeat[RH_FEMUR_ROLL,:].min(), target_joint_angles_repeat[RH_FEMUR_ROLL,:].argmin()
Rmax_femur_roll, Rmax_id_fem_roll = target_joint_angles_repeat[RH_FEMUR_ROLL,:].max(), target_joint_angles_repeat[RH_FEMUR_ROLL,:].argmax()

# plot femur pitch, min and max
plt.plot(target_joint_angles_repeat[RH_FEMUR,:], label='R Femur Pitch', color=Rfem_color_pitch)
plt.scatter(Rmin_id_fem_pitch, Rmin_femur_pitch, color=Rfem_color_pitch, label='Min 'f'{Rmin_id_fem_pitch:.2f}')
plt.scatter(Rmax_id_fem_pitch, Rmax_femur_pitch, color=Rfem_color_pitch, label='Max 'f'{Rmax_id_fem_pitch:.2f}')

# plot femur roll, min and max
plt.plot(target_joint_angles_repeat[RH_FEMUR_ROLL,:], label='R Femur Roll', color=Rfem_color_roll)
plt.scatter(Rmin_id_fem_roll, Rmin_femur_roll, color=Rfem_color_roll, label='Min 'f'{Rmin_id_fem_roll:.2f}')
plt.scatter(Rmax_id_fem_roll, Rmax_femur_roll, color=Rfem_color_roll, label='Max 'f'{Rmax_id_fem_roll:.2f}')

#### Tibia ####
Rtibia_color = 'lightblue'
Rmin_tibia_pitch, Rmin_id_tibia_pitch = target_joint_angles_repeat[RH_TIBIA,:].min(), target_joint_angles_repeat[RH_TIBIA,:].argmin()
Rmax_tibia_pitch, Rmax_id_tibia_pitch = target_joint_angles_repeat[RH_TIBIA,:].max(), target_joint_angles_repeat[RH_TIBIA,:].argmax()

# plot tibia pitch
plt.plot(target_joint_angles_repeat[RH_TIBIA,:], label='R Tibia Pitch', color=Rtibia_color)

# plot min & max of tibia pitch
plt.scatter(Rmin_id_tibia_pitch, Rmin_tibia_pitch, color=Rtibia_color, label='Min 'f'{Rmin_id_tibia_pitch:.2f}')
plt.scatter(Rmax_id_tibia_pitch, Rmax_tibia_pitch, color=Rtibia_color, label='Max 'f'{Rmax_id_tibia_pitch:.2f}')

#### Tarsus ####
Rtarsus_color = 'coral'
Rmin_tarsus_pitch, Rmin_id_tarsus_pitch = target_joint_angles_repeat[RH_TARSUS,:].min(), target_joint_angles_repeat[RH_TARSUS,:].argmin()
Rmax_tarsus_pitch, Rmax_id_tarsus_pitch = target_joint_angles_repeat[RH_TARSUS,:].max(), target_joint_angles_repeat[RH_TARSUS,:].argmax()

# plot tarsus pitch
plt.plot(target_joint_angles_repeat[RH_TARSUS,:], label='R Tarsus Pitch', color=Rtarsus_color)

plt.scatter(Rmin_id_tarsus_pitch, Rmin_tarsus_pitch, color=Rtarsus_color, label='Min 'f'{Rmin_id_tarsus_pitch:.2f}')
plt.scatter(Rmax_id_tarsus_pitch, Rmax_tarsus_pitch, color=Rtarsus_color, label='Max 'f'{Rmax_id_tarsus_pitch:.2f}')


plt.grid()
plt.title('Front Grooming: Leg Joints Angles')
plt.xlabel('Timestep (1 step = 0.001 s)')
plt.ylabel('Angle [rad]')
plt.legend()

plt.show()